## Set up trial

In [13]:
from Reporter import LightConcurrentReporter
from MarkovDPAI import *
import numpy as np
import pandas as pd

def perform_trial(parameters):
    path_penalty = parameters.get('path_penalty')
    loss_penalty = parameters.get('loss_penalty')
    score_factor = parameters.get('score_factor')
    num_trials = 1000

    ai_input=ExpectiMax8(depth = 6, path_pen=path_penalty, loss_penalty=loss_penalty, score_factor=score_factor, num_trials=num_trials)
    r = LightConcurrentReporter(ai_input, 50)
    report = pd.DataFrame(r.generate_report(), columns=['score','max_tile'])
    print(np.mean(report['score']), len(report[report['max_tile']>=4096])/50.0)
    return np.mean(report['score'])
   # return len(report[report['max_tile']>=4096])/50.0

### Begin optimization

In [15]:
from ax.service.ax_client import AxClient
from ax.modelbridge.registry import Models
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep

generation_strategy = GenerationStrategy(
    steps=[
        GenerationStep(model=Models.SOBOL, num_trials=10),
        GenerationStep(model=Models.GPEI, num_trials=-1),   # GPEI for the rest of the trials
    ]
)
ax_client = AxClient(generation_strategy=generation_strategy)
# ax_client = AxClient()
ax_client.create_experiment(name='optimize Params',
                            parameters=[
                                {'name': 'path_penalty',
                                    'type': 'range',
                                    'bounds': [0.0, 20.0]},
                                {'name': 'loss_penalty',
                                    'type': 'range',
                                    'bounds': [0.0, 20.0]},
                                {'name': 'score_factor',
                                    'type': 'range',
                                    'bounds': [0.0, 2.0]},
                            ],
)

for i in range(10):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))


for i in range(100):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))
    if i % 20 == 0:
        best_parameters, metrics = ax_client.get_best_parameters()
        print(f'The best parameters are: {best_parameters}')
        print(f'with score {metrics}')


best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

[INFO 11-08 23:51:03] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 11-08 23:51:03] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter path_penalty. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-08 23:51:03] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter loss_penalty. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-08 23:51:03] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter score_factor. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[IN

/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 11-08 23:51:03] ax.service.ax_client: Generated new trial 0 with parameters {'path_penalty': 9.468312, 'loss_penalty': 13.612159, 'score_factor': 1.777051} using model Sobol.
[INFO 11-08 23:57:11] ax.service.ax_client: Completed trial 0 with data: {'objective': (35135.76, None)}.
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 11-08 23:57:12] ax.service.ax_client: Generated new trial 1 with parameters {'path_penalty': 14.82108, 'loss_penalty': 5.668751, 'score_facto

35135.76 0.3


[INFO 11-08 23:57:43] ax.service.ax_client: Completed trial 1 with data: {'objective': (2608.24, None)}.
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 11-08 23:57:43] ax.service.ax_client: Generated new trial 2 with parameters {'path_penalty': 18.733325, 'loss_penalty': 17.787547, 'score_factor': 1.456911} using model Sobol.


2608.24 0.0


...

44253.28 0.52


[INFO 11-09 11:30:47] ax.service.ax_client: Generated new trial 108 with parameters {'path_penalty': 13.047386, 'loss_penalty': 0.0, 'score_factor': 2.0} using model GPEI.
[INFO 11-09 11:36:22] ax.service.ax_client: Completed trial 108 with data: {'objective': (31815.52, None)}.


31815.52 0.28


[INFO 11-09 11:36:24] ax.service.ax_client: Generated new trial 109 with parameters {'path_penalty': 1.506392, 'loss_penalty': 20.0, 'score_factor': 0.742039} using model GPEI.
[INFO 11-09 11:44:20] ax.service.ax_client: Completed trial 109 with data: {'objective': (52222.0, None)}.


52222.0 0.58
The best parameters are: {'path_penalty': 2.999628696289479, 'loss_penalty': 19.201009878081006, 'score_factor': 1.0465014951786142}
with score ({'objective': 51739.62863444732}, {'objective': {'objective': 3426711.0390681396}})


In [5]:
for i in range(120):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))
    if i % 20 == 0:
        best_parameters, metrics = ax_client.get_best_parameters()
        print(f'The best parameters are: {best_parameters}')
        print(f'with score {metrics}')
    
    
best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed on the second try, after generating a new set of initi

38339.44 0.28
The best parameters are: {'path_penalty': 5.696187845352844, 'loss_penalty': 10.0, 'score_factor': 1.9077351565843965, 'num_trials': 800}
with score ({'objective': 47256.1640477334}, {'objective': {'objective': 257828.74350236598}})


/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed on the second try, after generating a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
[INFO 11-08 07:43:50] ax.service.ax_client: Generated new trial 211 with parameters {'path_penalty': 5.13065, 'loss_penalty': 4.170804, 'score_factor': 2.0, 'num_trials': 800} using model GPEI.
[INFO 11-08 07:44:39] ax.service.ax_client: Completed trial 211 with d

49774.88 0.52


/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed on the second try, after generating a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
[INFO 11-08 07:44:52] ax.service.ax_client: Generated new trial 212 with parameters {'path_penalty': 5.741252, 'loss_penalty': 10.0, 'score_factor': 1.879704, 'num_trials': 0} using model GPEI.
[INFO 11

...

50604.16 0.52


/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
[INFO 11-08 09:42:05] ax.service.ax_client: Generated new trial 328 with parameters {'path_penalty': 3.27428, 'loss_penalty': 5.264745, 'score_factor': 1.081637, 'num_trials': 800} using model GPEI.
[INFO 11-08 09:42:49] ax.service.ax_client: Completed trial 328 with data: {'objective': (47110.8, None)}.


47110.8 0.48


/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
[INFO 11-08 09:43:04] ax.service.ax_client: Generated new trial 329 with parameters {'path_penalty': 3.634431, 'loss_penalty': 3.973014, 'score_factor': 1.134347, 'num_trials': 800} using model GPEI.
[INFO 11-08 09:43:48] ax.service.ax_client: Completed trial 329 with data: {'objective': (47763.04, None)}.


47763.04 0.48
The best parameters are: {'path_penalty': 3.2456769684027593, 'loss_penalty': 10.0, 'score_factor': 1.0632054172647503, 'num_trials': 800}
with score ({'objective': 47376.62136251005}, {'objective': {'objective': 232876.60807225382}})


In [ ]:
best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

data =  ax_client.experiment.fetch_data()
print(data.df)
print(data.df.sort_values(by='mean', ascending=False).head(10))

trial_params = ax_client.experiment.trials[42]
print(trial_params)

The best parameters are: {'path_penalty': 2.999628696289479, 'loss_penalty': 19.201009878081006, 'score_factor': 1.0465014951786142}
with score ({'objective': 51739.62863444732}, {'objective': {'objective': 3426711.0390681396}})
    arm_name metric_name      mean  sem  trial_index
0        0_0   objective  35135.76  NaN            0
1        1_0   objective   2608.24  NaN            1
2        2_0   objective  12360.96  NaN            2
3        3_0   objective  36827.92  NaN            3
4        4_0   objective  50389.92  NaN            4
..       ...         ...       ...  ...          ...
105    105_0   objective  48540.64  NaN          105
106    106_0   objective  46051.68  NaN          106
107    107_0   objective  44253.28  NaN          107
108    108_0   objective  31815.52  NaN          108
109    109_0   objective  52222.00  NaN          109

[110 rows x 5 columns]
   arm_name metric_name      mean  sem  trial_index
42     42_0   objective  59530.24  NaN           42
22     

KeyError: 290

In [ ]:
trial_params = ax_client.experiment.trials[42]
print(trial_params)
trial_params = ax_client.experiment.trials[22]
print(trial_params)
trial_params = ax_client.experiment.trials[49]
print(trial_params)

Trial(experiment_name='optimize Params', index=290, status=TrialStatus.COMPLETED, arm=Arm(name='290_0', parameters={'path_penalty': 3.3034937206974053, 'loss_penalty': 3.0689678841404167, 'score_factor': 1.1410012397734988, 'num_trials': 754}))
Trial(experiment_name='optimize Params', index=311, status=TrialStatus.COMPLETED, arm=Arm(name='311_0', parameters={'path_penalty': 3.3781955323633843, 'loss_penalty': 10.0, 'score_factor': 1.2050223824898296, 'num_trials': 0}))
Trial(experiment_name='optimize Params', index=149, status=TrialStatus.COMPLETED, arm=Arm(name='149_0', parameters={'path_penalty': 3.4846910016306003, 'loss_penalty': 10.0, 'score_factor': 1.1104202438519803, 'num_trials': 800}))
